**Reward Modelling**

In the second part of RLHF, i am showing how a model can be used as a value function by changing the last layer of the architecture. I use an example dataset from kaggle. Once we create our own data, we can use it as well ✌.

**File Upload**

The following block of code provides a mechanism to upload a file from your local machine to the Google Colab environment. Here, you're likely to upload your kaggle.json file. You can get your kaggle api token by logging into your kaggle account. Once you open your kaggle account, go to settings and then create a new kaggle api token. Then you upload the token by the code below to your google drive.

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

KeyboardInterrupt: ignored

**Installation of Required Libraries**

You're installing the necessary Python libraries for your task: transformers and datasets.

In [2]:
#!pip install datasets transformers[sentencepiece]
!pip install transformers
!pip install datasets

**Importing Dependencies**

After installing, you import the necessary modules from these libraries.

In [3]:
from datasets import load_dataset,Dataset,DatasetDict
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
import pandas as pd

**Setting Up Kaggle**

*   Next, you're creating a directory for Kaggle, copying the Kaggle API key (kaggle.json file) into it, and setting the correct permissions.
*   With Kaggle set up, you download the sarcasm detection dataset from Kaggle.
*   After downloading the dataset, you load it using the load_dataset function and perform various preprocessing operations.








In [26]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

data=load_dataset("json",data_files="/content/news-headlines-dataset-for-sarcasm-detection.zip")
data=data.rename_column("is_sarcastic","label")

data=data.remove_columns(['article_link'])

data.set_format('pandas')
data=data['train'][:]

data.drop_duplicates(subset=['headline'],inplace=True)
data=data.reset_index()[['headline','label']]
data=Dataset.from_pandas(data)

# 80% train, 20% test + validation
train_testvalid = data.train_test_split(test_size=0.2,seed=15)

# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5,seed=15)

# gather everyone if you want to have a single DatasetDict
data = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

data

mkdir: cannot create directory ‘/root/.kaggle’: File exists
news-headlines-dataset-for-sarcasm-detection.zip: Skipping, found more recently modified local copy (use --force to force download)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['headline', 'label'],
        num_rows: 22802
    })
    test: Dataset({
        features: ['headline', 'label'],
        num_rows: 2851
    })
    valid: Dataset({
        features: ['headline', 'label'],
        num_rows: 2850
    })
})

If you done these steps once, you can run the code below.

Reading File from Google Drive

You load dataset from your Google Drive, preprocess it, and split it into training and validation sets.

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
file_path = "/content/gdrive/My Drive/Sarcasm_Headlines_Dataset.json"
df = pd.read_json(file_path,lines=True)
data=df

In [28]:

data. rename(columns = {'is_sarcastic':'label'}, inplace = True)
data=data.drop('article_link',axis=1)

#data.set_format('pandas')
#data=data['train'][:]

#data.drop_duplicates(subset=['headline'],inplace=True)
#data=data.reset_index()[['headline','label']]
#data=Dataset.from_pandas(data)
#data

In [29]:
data.drop_duplicates(subset=['headline'],inplace=True)
data.reset_index(inplace=True)
data.drop(["index"],axis=1,inplace=True)
data

,headline,label
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0
...,...,...
26597,american politics in moral free-fall,0
26598,america's best 20 hikes,0
26599,reparations and obama,0
26600,israeli ban targeting boycott supporters raise...,0


In [30]:
df_train,df_val=train_test_split(data,test_size=0.1,random_state=42)

In [31]:

df_train.reset_index(inplace=True)
df_train.drop(["index"],axis=1,inplace=True)
df_train

,headline,label
0,"new monster energy defibrillator touts 1,200 v...",1
1,kim jong-il doesn't know how he keeps winning ...,1
2,frugal star wars fan camping out in front of 9...,1
3,cool mccain supporter wears 'mccain 2000' shir...,1
4,art major to stop capitalizing name,1
...,...,...
23936,"the 12 colleges with the biggest sweet tooth, ...",0
23937,"governor lashes out against cheap scotch, poor...",1
23938,14 photos that show the special bond between m...,0
23939,will smith says trump may force him to run for...,0


In [32]:
df_val.reset_index(inplace=True)
df_val.drop(["index"],axis=1,inplace=True)
df_val

,headline,label
0,departing employee not quite important enough ...,1
1,college student still managing to look like as...,1
2,fun sticker placed on child's ventilator,1
3,powerball officials remove plastic balls from ...,1
4,the roots to premiere 2 children's series on a...,0
...,...,...
2656,amazon shelves 'x-files' creator's sci-fi seri...,0
2657,congressman says new york city gunman got 'raw...,0
2658,city adds some big concrete stairs,1
2659,"i love my boyfriend, but i hate our relationsh...",0


In [33]:
#data=data.reset_index()[['headline','label']]
train_dataset=Dataset.from_pandas(df_train)
val_dataset=Dataset.from_pandas(df_val)
data = DatasetDict({
    'train': train_dataset,
    'valid': val_dataset})

data


#data=Dataset.from_pandas(data)
#data

DatasetDict({
    train: Dataset({
        features: ['headline', 'label'],
        num_rows: 23941
    })
    valid: Dataset({
        features: ['headline', 'label'],
        num_rows: 2661
    })
})

**Tokenizer and Model Initialization**

Here you initialize your tokenizer and your custom model. This model will be used for sequence classification.

In [34]:
checkpoint = "cardiffnlp/twitter-roberta-base-emotion"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.model_max_len=512

In [35]:
def tokenize(batch):
  return tokenizer(batch["headline"], truncation=True,max_length=512)

tokenized_dataset = data.map(tokenize, batched=True)
tokenized_dataset

Map:   0%|          | 0/23941 [00:00<?, ? examples/s]

Map:   0%|          | 0/2661 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['headline', 'label', 'input_ids', 'attention_mask'],
        num_rows: 23941
    })
    valid: Dataset({
        features: ['headline', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2661
    })
})

In [37]:
tokenized_dataset.set_format("torch",columns=["input_ids", "attention_mask", "label"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [38]:
class CustomModel(nn.Module):
  def __init__(self,checkpoint,num_labels):
    super(CustomModel,self).__init__()
    self.num_labels = num_labels

    #Load Model with given checkpoint and extract its body
    self.model = model = AutoModel.from_pretrained(checkpoint,config=AutoConfig.from_pretrained(checkpoint, output_attentions=True,output_hidden_states=True))
    self.dropout = nn.Dropout(0.1)
    self.classifier = nn.Linear(768,num_labels) # load and initialize weights

  def forward(self, input_ids=None, attention_mask=None,labels=None):
    #Extract outputs from the body
    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

    #Add custom layers
    sequence_output = self.dropout(outputs[0]) #outputs[0]=last hidden state

    logits = self.classifier(sequence_output[:,0,:].view(-1,768)) # calculate losses

    loss = None
    if labels is not None:
      loss_fct = nn.CrossEntropyLoss()
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

    return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states,attentions=outputs.attentions)

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=CustomModel(checkpoint=checkpoint,num_labels=2).to(device)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-emotion were not used when initializing RobertaModel: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-emotion and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

**Creating Dataloaders**

You create PyTorch Dataloaders for the training and evaluation datasets. These are used to feed data to the model in batches during training and evaluation.

In [40]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_dataset["train"], shuffle=True, batch_size=32, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_dataset["valid"], batch_size=32, collate_fn=data_collator
)

**Setting Up Optimizer and Learning Rate Scheduler**

Here you define the optimizer (AdamW) and the learning rate scheduler. The learning rate scheduler is used to modify the learning rate as the training progresses.

In [41]:
from transformers import AdamW,get_scheduler

#optimizer = AdamW(model.parameters(), lr=5e-5)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)


num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

2247


In [42]:
from datasets import load_metric
metric = load_metric("f1")

<ipython-input-42-e4b2221a8929>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1")


**Training and Evaluation**

The code performs the actual training and evaluation of the model.


tqdm is a Python library that provides a fast, extensible progress bar for loops and other iterable contexts. The name tqdm stands for "taqaddum" in Arabic which can mean "progress," and is a common pattern for Arabic four-letter words.

In [43]:
from tqdm.auto import tqdm

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epochs * len(eval_dataloader)))


for epoch in range(num_epochs):
  model.train()
  for batch in train_dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)
      loss = outputs.loss
      loss.backward()

      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
      progress_bar_train.update(1)

  model.eval()
  for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])
    progress_bar_eval.update(1)

  print(metric.compute())


  0%|          | 0/2247 [00:00<?, ?it/s]

  0%|          | 0/252 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'f1': 0.9229487713452726}
{'f1': 0.9041942604856511}
{'f1': 0.9229437229437231}


In [44]:
model.eval()

test_dataloader = DataLoader(
    tokenized_dataset["valid"], batch_size=32, collate_fn=data_collator
)

for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'f1': 0.9229437229437231}

In [45]:
model.eval()
test_dataloader = DataLoader(
    tokenized_dataset["valid"], batch_size=1, collate_fn=data_collator
)
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
print(predictions,batch["labels"])


tensor([0], device='cuda:0') tensor([0], device='cuda:0')


In [46]:
model.eval()

test_dataloader = DataLoader(
    tokenized_dataset["valid"], batch_size=32, collate_fn=data_collator
)

batch_counter = 0
for batch in test_dataloader:
    batch_counter += 1
    if batch_counter == 1:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        for i, prediction in enumerate(predictions):
            print("Model guess:", prediction.item(), "Real value:", batch["labels"][i].item())
    else:
        break

Model guess: 1 Real value: 1
Model guess: 1 Real value: 1
Model guess: 1 Real value: 1
Model guess: 1 Real value: 1
Model guess: 0 Real value: 0
Model guess: 0 Real value: 0
Model guess: 0 Real value: 0
Model guess: 1 Real value: 1
Model guess: 1 Real value: 1
Model guess: 0 Real value: 1
Model guess: 1 Real value: 1
Model guess: 0 Real value: 0
Model guess: 0 Real value: 0
Model guess: 0 Real value: 0
Model guess: 0 Real value: 0
Model guess: 1 Real value: 1
Model guess: 1 Real value: 1
Model guess: 0 Real value: 0
Model guess: 0 Real value: 0
Model guess: 0 Real value: 0
Model guess: 1 Real value: 1
Model guess: 0 Real value: 0
Model guess: 0 Real value: 0
Model guess: 0 Real value: 0
Model guess: 1 Real value: 1
Model guess: 1 Real value: 1
Model guess: 1 Real value: 1
Model guess: 0 Real value: 0
Model guess: 0 Real value: 0
Model guess: 0 Real value: 0
Model guess: 0 Real value: 0
Model guess: 1 Real value: 1
